In [4]:
import sys
sys.path.append("../")
import numpy as np
import os
import glob
import cv2
import csv
from scipy.signal import find_peaks
from utils import KymoMomentum, KymoTime, ErrorMeassgeCats
import pandas as pd
from tifffile import imread, imwrite
import matplotlib.pyplot as plt
from pathlib import Path


In [5]:
Kymo = '/home/kapoorlab/Downloads/Darine/Dpachys_10/DistanceMethodCurvature_Dpachys_controlTrackID1.tif'
savedir = '/home/kapoorlab/Downloads/'
try:
  Path(savedir).mkdir(exist_ok = True)
except :
   ErrorMeassgeCats()
Time_unit = 5
Xcalibration = 0.212
#Remove zero end of kymograph
cutEnd = 300
#This is what makes a peak a peak, the height of peaks around this main peak height threshold
Heightthreshold = 0.005



https://res.cloudinary.com/twenty20/private_images/t_reshot-300/v1603751579/photosp/0b64dafe-b321-4529-9476-3df53d820aac/0b64dafe-b321-4529-9476-3df53d820aac.jpg
You have an error in the code, enjoy looking at this picture and contact your ground control to major Tom


In [ ]:
Kymoimage = imread(Kymo)
Kymoimage = Kymoimage[:Kymoimage.shape[0]-cutEnd,:]
Name = os.path.basename(os.path.splitext(Kymo)[0])
peakKX = KymoMomentum(Kymoimage,Xcalibration,savedir, threshold = Heightthreshold)

peakWT = KymoTime(Kymoimage, Time_unit,savedir, threshold = Heightthreshold)

Velocity = 0
if peakKX > 0:
 Velocity = peakWT/peakKX

print("Estimated Velocity", Velocity, 'um/s')

df = pd.DataFrame([[Name,Velocity]],columns =['Experiment', 'Velocity'])
df.to_csv(savedir + Name + 'Velocity' +  '.csv', index = False)  